In [1]:
import cv2
import socket

import numpy as np
from time import sleep

server_ip = "10.0.0.26"
server_port = 1024
display_width = 128
display_height = 64
scale = 2

display_buffer_size = display_width * display_height // 8

def bitmap_binary_to_rgb(binary: bytes):
    buffer = bytearray(display_width * display_height * 3)
    x = 0
    y = 0

    while x < display_width:
        while y < display_height:
            bit = binary[x + (y // 8) * display_width] & (1 << (y % 8))
            buffer[(x + (y * display_width)) * 3] = 255 if bit else 0
            buffer[(x + (y * display_width)) * 3 + 1] = 255 if bit else 0
            buffer[(x + (y * display_width)) * 3 + 2] = 255 if bit else 0
            y += 1

        x += 1
        y = 0

    return bytes(buffer)

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

while True:
    try:
        sock.connect((server_ip, server_port))
        print("Connected to server")

        while True:
            sock.send(b"\x00")
            data = np.frombuffer(bitmap_binary_to_rgb(sock.recv(display_buffer_size)), dtype=np.uint8)
            data = data.reshape((display_height, display_width, 3))
            data = cv2.resize(data, (display_width * scale, display_height * scale), interpolation=cv2.INTER_NEAREST)
            
            cv2.imshow("Display", data)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            sleep(1 / 144)
    except KeyboardInterrupt:
        pass
    except Exception as e:
        print(e)
        sock.close()
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)



Connected to server
index out of range
Connected to server
index out of range
Connected to server
index out of range
Connected to server
